# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,grytviken,-54.2811,-36.5092,31.12,91,57,11.61,GS,1701986520
1,1,yeppoon,-23.1333,150.7333,78.58,64,37,17.78,AU,1701986810
2,2,lihue,21.9789,-159.3672,80.37,71,75,8.05,US,1701986678
3,3,muisne,0.6000,-80.0333,78.57,85,78,8.19,EC,1701986810
4,4,pacific grove,36.6177,-121.9166,62.42,69,20,6.91,US,1701986707


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles ="OSM",
size = "Humidity",
color = "City",
frame_width = 700,
frame_height= 500)

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df
ideal_city_data_df = ideal_city_data_df.loc[(ideal_city_data_df["Max Temp"] < 27) & (ideal_city_data_df["Max Temp"] > 21)]
ideal_city_data_df = ideal_city_data_df.loc[(ideal_city_data_df["Windspeed"] < 10)]



# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
34,34,garwolin,51.8975,21.6147,26.13,95,79,7.36,PL,1701986813
47,47,ilulissat,69.2167,-51.1000,26.62,63,100,0.00,GL,1701986814
103,103,thompson,55.7435,-97.8558,21.76,96,100,1.57,CA,1701986821
124,124,naryn,41.4287,75.9911,21.20,55,42,8.46,KG,1701986825
157,157,sisimiut,66.9395,-53.6735,23.00,70,100,6.87,GL,1701986657
180,180,vilyuchinsk,52.9306,158.4028,21.47,79,100,3.80,RU,1701986834
189,189,smithers,54.7804,-127.1743,25.12,93,100,5.75,CA,1701986610
200,200,la sarre,48.8002,-79.1996,25.18,92,100,3.80,CA,1701986837
248,248,peace river,56.2501,-117.2860,26.91,80,20,3.44,CA,1701986595
345,345,campbellton,48.0075,-66.6727,21.87,82,44,7.29,CA,1701986857


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
34,garwolin,PL,51.8975,21.6147,95,
47,ilulissat,GL,69.2167,-51.1000,63,
103,thompson,CA,55.7435,-97.8558,96,
124,naryn,KG,41.4287,75.9911,55,
157,sisimiut,GL,66.9395,-53.6735,70,
180,vilyuchinsk,RU,52.9306,158.4028,79,
189,smithers,CA,54.7804,-127.1743,93,
200,la sarre,CA,48.8002,-79.1996,92,
248,peace river,CA,56.2501,-117.2860,80,
345,campbellton,CA,48.0075,-66.6727,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "radius": radius,
    "apiKey" : geoapify_key
    
}
 


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{long},{lat},{radius}'
    params["bias"] = f'proximity:{long},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
garwolin - nearest hotel: Hotel Elmar
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
thompson - nearest hotel: Thompson Inn
naryn - nearest hotel: Ала-Тоо
sisimiut - nearest hotel: Sømandshjemmet
vilyuchinsk - nearest hotel: Победа
smithers - nearest hotel: Sunshine Inn Hotel
la sarre - nearest hotel: Motel le Bivouac
peace river - nearest hotel: New Ridge Inn
campbellton - nearest hotel: Campbellton City Center Mall
falun - nearest hotel: First Hotel Grand
egilsstadir - nearest hotel: Valaskjálf


,City,Country,Lat,Lng,Humidity,Hotel Name
34,garwolin,PL,51.8975,21.6147,95,Hotel Elmar
47,ilulissat,GL,69.2167,-51.1000,63,Best Western Plus Hotel Ilulissat
103,thompson,CA,55.7435,-97.8558,96,Thompson Inn
124,naryn,KG,41.4287,75.9911,55,Ала-Тоо
157,sisimiut,GL,66.9395,-53.6735,70,Sømandshjemmet
180,vilyuchinsk,RU,52.9306,158.4028,79,Победа
189,smithers,CA,54.7804,-127.1743,93,Sunshine Inn Hotel
200,la sarre,CA,48.8002,-79.1996,92,Motel le Bivouac
248,peace river,CA,56.2501,-117.2860,80,New Ridge Inn
345,campbellton,CA,48.0075,-66.6727,82,Campbellton City Center Mall


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
map1 = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles ="OSM",
size = "Humidity",
color = "City",
hover_cols=["Hotel Name", "Country"],
frame_width = 700,
frame_height= 500)


map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)